# label adapter FCN

In [1]:
import torchvision.transforms as T
import torchvision.transforms.functional as F
import albumentations as albu
import random
import torch
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt
import segmentation_models_pytorch as smp
from segmentation_models_pytorch import utils
import os
import matplotlib.pyplot as plt
import label_test_script

2023-02-17 21:06:10.607192: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-17 21:06:10.940951: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-17 21:06:12.213213: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/nathan/miniconda3/lib/python3.9/site-packages/cv2/../../lib64::/home/nathan/miniconda3/lib/:/home/nathan/miniconda3/lib/:/home/nathan/miniconda3/lib/
2023-02-17 21:06:12.213465: W tensorflow/stream_executor

## Initialise Unet

In [2]:
#Hyperparamters
ENCODER = 'resnet101'
ENCODER_WEIGHTS = 'imagenet' #pretrained weighting
ACTIVATION = "sigmoid" # softmax2d for multiclass segmentation
num_classes = 11

mobile = smp.Unet(
    in_channels=3,
    encoder_name="mobilenet_v2", 
    encoder_weights=ENCODER_WEIGHTS, 
    classes=num_classes, 
    activation=ACTIVATION,
    decoder_use_batchnorm = True,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

## Preprocessing and DS

In [3]:
def get_training_augmentation():
    train_transform = [
        albu.Rotate((-11,11),p=0.5),
        albu.PadIfNeeded(min_height=128, min_width=128, always_apply=True, border_mode=0),
        albu.Perspective(p=0.4),
    ]

    return albu.Compose(train_transform)


def transformation_augs():
    train_transform = [
    ]
    return albu.Compose(train_transform)


def get_validation_augmentation():
    """Add paddings to make image shape divisible by 32"""
    test_transform = [
        albu.PadIfNeeded(128, 128)
    ]
    return albu.Compose(test_transform)

class MyDataSet(torch.utils.data.Dataset):

  def __init__(self, images_dir, masks_dir, coords_dir, preprocessing=None, classes=None,augmentation=None, training=True):
    super(MyDataSet, self).__init__()
    
    self.preprocessing = preprocessing
    self.augmentation = augmentation

    self.image_ids = os.listdir(images_dir)

    train_slice = int(len(self.image_ids)*0.2)

    if training == True:
      self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.image_ids][train_slice::]
      self.masks_fps = [os.path.join(masks_dir, mask_id) for mask_id in self.image_ids][train_slice::]
    else:
      self.images_fps = [os.path.join(images_dir, image_id) for image_id in self.image_ids][:train_slice]
      self.masks_fps = [os.path.join(masks_dir, mask_id) for mask_id in self.image_ids][:train_slice]

  def __len__(self):
    # a DataSet must know it size
    return len(self.images_fps)

  def __getitem__(self, i):

    image = cv2.imread(self.images_fps[i], 0)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    mask = cv2.imread(self.masks_fps[i], cv2.IMREAD_GRAYSCALE)

    sample = self.augmentation(image=image, mask=mask)
    image, mask = sample['image'], sample['mask']

    image = torch.tensor(image)

    #onehot
    one_hot_Y = torch.nn.functional.one_hot(torch.tensor(mask).to(torch.int64), 11).float()

    return (image.permute(2,0,1).float(), one_hot_Y.permute(2,0,1).float())


train_x_path = "/home/nathan/Documents/final_project/datasets/label_adapter_helen_train/images"
train_y_path = "/home/nathan/Documents/final_project/datasets/label_adapter_helen_train/masks"

val_x_path = train_x_path
val_y_path = train_y_path

train_ds2 = MyDataSet(train_x_path, train_y_path, None, augmentation=get_training_augmentation())
val_ds2 = MyDataSet(val_x_path, val_y_path, None, training=False, augmentation=get_validation_augmentation())


In [4]:
from torch.utils.data import DataLoader

# Get train and val data loaders
train_loader2 = DataLoader(train_ds2, batch_size=24, shuffle=True, num_workers=7)
valid_loader2 = DataLoader(val_ds2, batch_size=24, shuffle=False, num_workers=7)

## Training

In [5]:
# Set flag to train the model or not. If set to 'False', only prediction is performed (using an older model checkpoint)
TRAINING = True
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loss = smp.utils.losses.CrossEntropyLoss()

# define evalusation metrics
metrics = [
    smp.utils.metrics.IoU(threshold=0.5),
]

In [6]:
def train_model(model, train_loader_given, valid_loader_given, saveas, epoch):

    #clear CUDA cache
    torch.cuda.empty_cache()

    # define optimizer
    optimizer = torch.optim.NAdam(model.parameters(), lr=0.001)

    # define learning rate scheduler
    #lr_scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer)

    #define train ecpochs w/ our hyperparams
    train_epoch = smp.utils.train.TrainEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        optimizer=optimizer,
        device=DEVICE,
        verbose=True,
    )

    #define train ecpochs w/ our hyperparams
    valid_epoch = smp.utils.train.ValidEpoch(
        model, 
        loss=loss, 
        metrics=metrics, 
        device=DEVICE,
        verbose=True,
    )

    if TRAINING:

        best_iou_score = 0.0
        train_logs_list, valid_logs_list = [], []

        #Keep track of epoch BCE and IoU for graph plots
        epoch_count = []
        v_bce = []
        v_ious = []

        t_bce = []
        t_ious = []

        i = 0
        while i < epoch:
            i +=1 

            # Perform training & validation
            print('\nEpoch: {}'.format(i))
            train_logs = train_epoch.run(train_loader_given)
            valid_logs = valid_epoch.run(valid_loader_given)
            #lr_scheduler.step()

            #log
            train_logs_list.append(train_logs)
            valid_logs_list.append(valid_logs)

            # Save model if a better val IoU score is obtained
            if best_iou_score < valid_logs['iou_score']:
                best_iou_score = valid_logs['iou_score']
                torch.save(model, saveas)
                print('Model saved!')

            v_ious.append(valid_logs['iou_score'])
            t_ious.append(train_logs['iou_score'])
            
            epoch_count.append(i)
        
            # Plot graph every 2 epochs
            if i % 2 ==0:
                plt.show()
                plt.plot(epoch_count,t_ious, label="Train IoU")
                plt.plot(epoch_count,v_ious, label="Valid IoU")
                plt.legend(loc="upper left")
                plt.xlabel("Epochs")
                plt.ylabel("IoU")
                plt.show()


In [7]:
#train_model(mobile, train_loader2, valid_loader2, "/home/nathan/Documents/final_project/saved_models/label_adapted_helen.pth", 50)